This notebook runs skeletonization on segmentation in a zarr or npy format using the tensorstore package.

In [ ]:
import numpy

from ac_segmentation.postprocess.skeletonize_array import *
from ac_segmentation.utils.io import write_cv_skels_iter_tar
from ac_segmentation.utils.tensorstore import open_ZarrTensor, create_EmptyTensor

In [ ]:
#Open array from zarr file or gzip npy file
prob_map = open_ZarrTensor("/ACdata/Users/connorl/Image_Files/TS_Array_S32_Pos56.zarr/", bytes_limit= 100_000_000)
#prob_map = read_gzip_array(probability_input_path,preprocess_func=lambda x: (x.astype(numpy.float32) / 255.))

In [ ]:
#Set skeletonization parameters
prob_threshold = .05
size_threshold = 80
chunk_size=[1000,1000,1000]
skeleton_output_path = "./test.swc.gz"

In [ ]:
#Run skeletonization
binary_arr = threshold_binarize_array(prob_map, threshold=prob_threshold )
labeled_arr, _ = label_binary_array(binary_arr, size_threshold=size_threshold)
%time skels = skeletonize_labeled_array_concurrent(labeled_arr, chunk_size=chunk_size, n_jobs=10)

In [ ]:
#Write neurons to swc tar file
write_cv_skels_iter_tar(skeleton_output_path, skels)